# 버스정류장 전처리
* 서울시 공공데이터포털에서 'BUS_STATION_BOARDING_MONTH_202107.csv' 다운(21년 7월 버스정류소별 승하차인원 정보)
* 서울시 공공데이터포털에서 '서울시버스정류소좌표데이터.csv) 다운
* 서대문구 내 버스정류장 번호(텍스트파일)
* 결론 컬럼 -> '버스정류장번호', '정류소명', 'X좌표', 'Y좌표', '승차총승객수', '하차총승객수', '승하차총승객수'


In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
# 서대문구 내 버스정류장 
url = './데이터 원본/서대문구 버스정류장.xlsx'
bus_df = pd.read_excel(url)

# 버스정류장번호를 시리즈로 저장
bus_num = bus_df['버스정류장번호']
bus_num

0      13911
1      13028
2      13029
3      13802
4      13021
       ...  
452    13882
453    13930
454    13902
455    13907
456    13859
Name: 버스정류장번호, Length: 457, dtype: int64

## 버스정류장별 승하차인원

In [3]:
url = './데이터 원본/BUS_STATION_BOARDING_MONTH_202107.csv'
bus_df_1 = pd.read_csv(url, encoding='cp949')
bus_df_1.head(3)

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20210701,100,100번(하계동~용산구청),11428,한성여객종점,21,1,20210704
1,20210701,100,100번(하계동~용산구청),11387,노원평생학습관,13,0,20210704
2,20210701,104,104번(강북청소년수련관난나~숭례문),09121,인수동장미원,42,146,20210704


In [4]:
# 필요한 칼럼만
bus_df_2 = bus_df_1[['버스정류장ARS번호', '승차총승객수', '하차총승객수']]
# 칼럼명 변환
bus_df_2 = bus_df_2.rename(columns={'버스정류장ARS번호' : '버스정류장번호'})
bus_df_2

,버스정류장번호,승차총승객수,하차총승객수
0,11428,21,1
1,11387,13,0
2,09121,42,146
3,09110,57,132
4,09197,0,134
...,...,...,...
1191177,08304,301,90
1191178,14201,117,15
1191179,14220,149,974
1191180,14152,102,83


In [5]:
# 버스정류장 번호가 ~이 아닌 것들만
bus_df_3 = bus_df_2[bus_df_2['버스정류장번호'] != '~']
bus_df_3 = bus_df_3.sort_values('버스정류장번호')
bus_df_3.index = range(len(bus_df_3))
bus_df_3

,버스정류장번호,승차총승객수,하차총승객수
0,01001,6,9
1,01001,126,157
2,01001,4,7
3,01001,3,8
4,01001,5,14
...,...,...,...
1185918,68352,13,62
1185919,68352,19,85
1185920,68352,16,63
1185921,68352,17,60


In [6]:
# 버스정류장번호의 데이터타입이 다르므로 데이터타입 바꿔주기
print(type(bus_df_3['버스정류장번호'][0]))
print(type(bus_num[0]))
bus_num = bus_num.astype('str')
print(type(bus_num[0]))

<class 'str'>
<class 'numpy.int64'>
<class 'str'>


In [7]:
# 서대문구 버스정류장만 파씽
col = ['버스정류장번호', '승차총승객수', '하차총승객수']
bus_df_4 = pd.DataFrame(columns=col)

for i in tqdm(range(len(bus_num))):
    temp_df = bus_df_3[bus_df_3['버스정류장번호'] == bus_num[i]]
    up = temp_df['승차총승객수'].sum()
    down = temp_df['하차총승객수'].sum()
    total = up + down
    
    new_row = {'버스정류장번호' : bus_num[i], '승차총승객수' : up, '하차총승객수' : down}
    bus_df_4 = bus_df_4.append(new_row, ignore_index=True)
    
bus_df_4

100%|████████████████████████████████████████████████████████████████████████████████| 457/457 [00:32<00:00, 14.21it/s]


,버스정류장번호,승차총승객수,하차총승객수
0,13911,114636,80588
1,13028,108947,109220
2,13029,108056,87437
3,13802,106088,47311
4,13021,99108,52219
...,...,...,...
452,13882,8,3885
453,13930,11,1505
454,13902,10,218
455,13907,2,913


In [8]:
total = []
for i in range(len(bus_df_4)):
    total.append(bus_df_4['승차총승객수'][i] + bus_df_4['하차총승객수'][i])

bus_df_4['승하차총승객수'] = total
bus_df_4

,버스정류장번호,승차총승객수,하차총승객수,승하차총승객수
0,13911,114636,80588,195224
1,13028,108947,109220,218167
2,13029,108056,87437,195493
3,13802,106088,47311,153399
4,13021,99108,52219,151327
...,...,...,...,...
452,13882,8,3885,3893
453,13930,11,1505,1516
454,13902,10,218,228
455,13907,2,913,915


## 주요 버스정류장 좌표 추가

In [9]:
url = './데이터 원본/서울시버스정류소좌표데이터.csv'
bus_coordinate_df = pd.read_csv(url, encoding='cp949')
bus_coordinate_df = bus_coordinate_df.rename(columns={'ARS-ID' : '버스정류장번호', 'X좌표' : '경도', 'Y좌표' : '위도'})
bus_coordinate_df

,표준ID,버스정류장번호,정류소명,경도,위도
0,100000001,1001,종로2가사거리,126.987786,37.569764
1,100000002,1002,창경궁.서울대학교병원,126.996520,37.579179
2,100000003,1003,명륜3가.성대입구,126.998290,37.582709
3,100000004,1004,종로2가.삼일교,126.987507,37.568582
4,100000005,1005,혜화동로터리.여운형활동터,127.001694,37.586230
...,...,...,...,...,...
10916,124000334,25995,우성아파트,127.139326,37.550442
10917,124000333,25996,우성아파트,127.140052,37.550620
10918,124000332,25997,조일약국,127.123477,37.533686
10919,124000331,25998,성내시장,127.125410,37.536199


In [10]:
# 각 버스정류장번호의 데이터타입이 다르므로 데이터타입 바꿔주기
print(type(bus_coordinate_df['버스정류장번호'][0]))
print(type(bus_num[0]))
bus_num = bus_num.astype('int64')
print(type(bus_num[0]))

<class 'numpy.int64'>
<class 'str'>
<class 'numpy.int64'>


In [11]:
bus_df_5 = bus_df_4.copy()
bus_df_5

,버스정류장번호,승차총승객수,하차총승객수,승하차총승객수
0,13911,114636,80588,195224
1,13028,108947,109220,218167
2,13029,108056,87437,195493
3,13802,106088,47311,153399
4,13021,99108,52219,151327
...,...,...,...,...
452,13882,8,3885,3893
453,13930,11,1505,1516
454,13902,10,218,228
455,13907,2,913,915


In [12]:
# bus_df_5에 경도, 위도, 정류소명 추가
name = []
lat = []
lng = []


for i in tqdm(range(len(bus_num))):
    temp_df = bus_coordinate_df[bus_coordinate_df['버스정류장번호'] == bus_num[i]]
    name.append(temp_df['정류소명'].values[0])
    lat.append(temp_df['위도'].values[0])
    lng.append(temp_df['경도'].values[0])
    
    
bus_df_5['정류소명'] = name
bus_df_5['경도'] = lng
bus_df_5['위도'] = lat
    
bus_df_5

100%|██████████████████████████████████████████████████████████████████████████████| 457/457 [00:00<00:00, 1642.13it/s]


,버스정류장번호,승차총승객수,하차총승객수,승하차총승객수,정류소명,경도,위도
0,13911,114636,80588,195224,홍제역.3번출구,126.944321,37.588500
1,13028,108947,109220,218167,홍제역,126.944350,37.588479
2,13029,108056,87437,195493,홍제역,126.945368,37.587879
3,13802,106088,47311,153399,신촌전철역,126.937262,37.555450
4,13021,99108,52219,151327,신촌오거리.현대백화점,126.934527,37.556026
...,...,...,...,...,...,...,...
452,13882,8,3885,3893,홍은2동어린이집앞,126.934155,37.579578
453,13930,11,1505,1516,인왕산현대아파트101동,126.950416,37.586198
454,13902,10,218,228,홍은동국민주택,126.949877,37.605195
455,13907,2,913,915,홍은동국민주택,126.949971,37.605549


In [13]:
bus_df_5.to_excel('2. 버스정류장_전처리.xlsx')